In [1]:
import pandas as pd 
import numpy as np 
import re
import torch

In [5]:
%cd KoBERT

d:\5.학교 백업\4학년 1학기\사이버보안 졸업 프로젝트\Recommender\KoBERT


In [6]:
%pwd

'd:\\5.학교 백업\\4학년 1학기\\사이버보안 졸업 프로젝트\\Recommender\\KoBERT'

In [2]:
case=pd.read_csv('case_list.csv')
case.head(5)

,index,datanumber,title,casenumber,date,verdicttype,holding,summary,jmreference,prreference,prny
0,0,230961,주주총회소집허가,2022마5372,20220907,결정,소수주주가 상법 제366조에 따라 임시총회 소집에 관한 법원의 허가를 신청할 때...,일반적으로 주주총회는 이사회의 결의로 소집하지만(상법 제362조)$ 예외적으로 ...,상법 제361조$ 제362조$ 제366조$ 제385조 제1항$ 제389조$ 민사소송...,대법원 2022. 4. 19. 자 2022그501 결정(공2022상$ 1013),【신청인$ 재항고인】 신청인 (소송대리인 변호사 오원근)【사건본인$ 상대방】 주식회...
1,1,230971,소유권이전등기,2022다217117,20220907,판결,[1] 수임인이 위임인을 위하여 자기 명의로 취득한 권리를 위임인에게 이전하여야...,"[1] 민법 제684조 제2항은 ""수임인이 위임인을 위하여 자기의 명의로 취득한...",[1] 민법 제684조 / [2] 민법 제684조,[1] 대법원 2007. 2. 8. 선고 2004다64432 판결(공2007상$ ...,【원고$ 상고인 겸 피상고인】 하동지구개발사업단 주식회사의 파산관재인 (소송대리인 ...
2,2,230967,폭력행위등처벌에관한법률위반(단체등의구성·활동)·폭력행위등처벌에관한법률위반(단체등의공...,2022도6993,20220907,판결,[1] 공소장변경이 허용되는 범위 / 공소사실의 동일성이 인정되지 않는 범죄사실...,[1] 공소장변경은 공소사실의 동일성이 인정되는 범위 내에서만 허용되고$ 공소사...,[1] 형사소송법 제298조 제1항 / [2] 폭력행위 등 처벌에 관한 법률 제4...,[1] 대법원 1994. 3. 22. 선고 93도2080 전원합의체 판결(공199...,【피 고 인】 피고인 1 외 1인【상 고 인】 피고인들【변 호 인】 변호사 현희철 ...
3,3,230957,연차수당지급,2022다245419,20220907,판결,1년을 초과하되 2년 이하의 기간 동안 근로를 제공한 근로자에게 부여될 수 있는...,근로기준법에 따르면$ 사용자는 1년간 80% 이상 출근한 근로자에게 15일의 연...,근로기준법 제60조 제1항$ 제2항,대법원 2018. 6. 28. 선고 2016다48297 판결(공2018하$ 1435...,【원고$ 상고인】 성창산업개발 주식회사【피고$ 피상고인】 재단법인 오송첨단의료산업진...
4,4,230981,중재판정의집행,2020마5970,20220907,결정,중재요청서 등의 서면 통지에 있어서 중재법 제4조 제3항의 발송에 의한 통지가 ...,중재절차는 당사자 간에 다른 합의가 없는 한 피신청인이 중재요청서를 받은 날부터...,중재법 제4조$ 제22조 제1항$ 제36조 제2항 제1호 (나)목$ 제38조 제1호...,-,【신청인$재항고인】 주식회사 포스코아이씨티 (소송대리인 법무법인(유한) 화우 담당변...


In [40]:
#case prny를 가공해서 kobert 만들기
def preprocess(passage):
    stopwords='[○\(\)\[\]\{\}\$\【\】\,,외,선고,법,원고탈퇴,피,]'
    passed=re.sub(stopwords,'',passage)
    passed=re.sub('피 고 인','피고인',passed)
    passed=re.sub('호 소 인','호소인',passed)
    passed=re.sub('피  인','피인',passed)
    passed=re.sub('호 인','호인',passed)
    passed=re.sub('상 ',' ',passed)
    passed=re.sub('겸 ',' ',passed)
    passed=re.sub(' 인',' ',passed)
    passed=re.sub('[ ]+',' ',passed)
    passed=re.sub('[가-힣]+\.','*',passed)
    passed=re.sub('\.','',passed)
    return passed
prny=case.copy()['prny'].apply(preprocess)
prny

0     신청인 재항인 신청인 소송대리인 변호사 오근사건본인 상대방 주식회사 제나코리아심결정...
1      상인 상인 하동지구개발사업단 주식회사의 파산관재인 소송대리인 무인유한 바른 담당변...
2      1 1인 들변 호인 변호사 현희철 1인심판결 대전 2022 5 24 2021노50...
3      상인 성창산업개발 주식회사 상인 재단인 오송첨단의료산업진흥재단 소송대리인 무인 청...
4     신청인재항인 주식회사 포스코아이씨티 소송대리인 무인유한 화우 담당변호사 최영관 1인...
5      상인 한솔1호발전소 주식회사 상인 소송대리인 변호사 이경익심판결 서울 2022 4...
6      1 1인 들변 호인 변호사 윤길웅 1인심판결 부산 2022 6 22 2021노49...
7      1 3인들 승계참가인 상인 교보자산신탁주식회사 상인 소송대리인 무인 대경종합률사무...
8      상인 칸서스자산운용 주식회사 소송대리인 무인유한 지평 담당변호사 배기완 2인 상인...
9      상인 상인 삼성화재해상보험 주식회사 소송대리인 무인유한 지평 담당변호사 김영수 2...
10     상인 대한예수교장로회 교회 소송대리인 무인 다비다 담당변호사 조동섭 상인 소송대리...
11     상인 보조참가인 상인 보조참가인 소송대리인 무인유한 광장 담당변호사 우람찬 1인심...
12     상인 에스케이텔레콤 주식회사 소송대리인 무인유한 태평양 2인 상인 남대문세무서장 ...
13     심판결 부산 2020 1 9 2019노472 판결주 문 상를 * 이 유 상이유를 ...
14     상인 소송대리인 특허인 태웅 담당변리사 이태림 상인 특허청장심판결 특허 2020 ...
15     심판결 수지 2019 5 3 2018노6585 판결주 문 심판결을 파기하 사건을 ...
16     상인 별지 명단 기재와 * 소송대리인 무인 덕수 담당변호사 김형태 1인 상인 대한...
17     및 검사변 호인 무인 여의 담당변호사 오영신 1인심판결 서울서부지 2020

In [41]:
def bert_input_fy(passage):
    return passage.split('*')

bert_input=[]
for p in prny:
    bert_input+=bert_input_fy(p)

In [23]:
%cd KoBERT
%ls

d:\PanryeAI\KoBERT
 D ����̺��� ����: �� ����
 ���� �Ϸ� ��ȣ: A467-E339

 d:\PanryeAI\KoBERT ���͸�

2022-12-09  ���� 04:44    <DIR>          .
2022-12-09  ���� 04:44    <DIR>          ..
2022-12-09  ���� 04:44    <DIR>          .git
2022-12-09  ���� 04:44    <DIR>          .github
2022-11-01  ���� 12:52                 8 .gitignore
2022-12-09  ���� 04:44    <DIR>          imgs
2022-12-09  ���� 04:44    <DIR>          kobert
2022-12-09  ���� 04:44    <DIR>          kobert_hf
2022-11-01  ���� 12:52            11,547 LICENSE
2022-12-09  ���� 04:44    <DIR>          logs
2022-11-01  ���� 12:52             9,964 README.md
2022-11-01  ���� 12:52               200 requirements.txt
2022-12-09  ���� 04:44    <DIR>          scripts
2022-11-01  ���� 12:52               720 setup.py
               5�� ����              22,439 ����Ʈ
               9�� ���͸�  90,138,210,304 ����Ʈ ����


In [35]:
#try 1
from kobert import get_tokenizer
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [42]:
import torch
from transformers import BertModel
model = BertModel.from_pretrained('skt/kobert-base-v1')
#inputs = tokenizer.batch_encode_plus([text])
inputs=tokenizer.batch_encode_plus(bert_input,pad_to_max_length=True)

c:\Users\Heeyo\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\tokenization_utils_base.py:2212: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


## checking out input

In [43]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [51]:
len(inputs['input_ids'][2])#-len(set(inputs['input_ids'][0]))

283

In [48]:
max(list(map(len,inputs['input_ids'])))

283

In [45]:
print(inputs['input_ids'][0])

[2, 3030, 7119, 3969, 7846, 7119, 3030, 7119, 2839, 5808, 6122, 7119, 2345, 3417, 5546, 6495, 6383, 7119, 2664, 6305, 4230, 7957, 4128, 5655, 7535, 6745, 5417, 1660, 554, 134, 553, 538, 3886, 554, 133, 6003, 119, 141, 945, 7276, 2120, 3060, 5417, 4257, 2348, 7318, 2071, 1258, 7191, 4128, 93, 7925, 517, 7095, 6812, 2437, 7088, 4799, 5561, 7782, 3647, 2437, 2575, 1660, 5944, 6896, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [52]:
print(inputs['input_ids'][3])

[2, 4279, 2254, 7283, 7452, 6629, 7095, 535, 6416, 7095, 589, 3704, 6896, 5001, 4230, 7088, 769, 4241, 5760, 5160, 7095, 2076, 6513, 5468, 2849, 7095, 3735, 4005, 2718, 6198, 7117, 712, 7136, 6705, 7454, 6607, 7354, 7431, 5495, 6553, 498, 6003, 4965, 7088, 3703, 6896, 4156, 7815, 3831, 7454, 7095, 2849, 7088, 4484, 7836, 2872, 3854, 3552, 4484, 5330, 3854, 7086, 5176, 4297, 7436, 3283, 517, 7454, 6607, 7354, 7095, 4071, 5330, 4370, 5898, 3162, 1844, 6900, 517, 7095, 5038, 6116, 2226, 4358, 4512, 7955, 2849, 7836, 2872, 3862, 4128, 142, 198, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

## training model

In [46]:
out = model(input_ids = torch.tensor(inputs['input_ids']),
              attention_mask = torch.tensor(inputs['attention_mask']))
out.pooler_output.shape

RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:81] data. DefaultCPUAllocator: not enough memory: you tried to allocate 4636192032 bytes.

In [29]:
torch.save(out.pooler_output, '../dataset/case_nm_tensor.pt')

NameError: name 'out' is not defined